In [ ]:
#!pip install recordlinkage==0.14
import recordlinkage
import pandas as pd
import recordlinkage as rl 
from recordlinkage.index import Block
from recordlinkage.algorithms.distance import _haversine_distance
from recordlinkage.algorithms.numeric import _linear_sim
import random
import pickle 
import numpy as np

In [ ]:
#Reading Data
df_clean = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_red.csv')
#There was a duplicated entry in ListingId so delete
df_clean[df_clean.duplicated(subset=['ListingId']) == True]
df_clean.drop(132,axis=0,inplace=True)
df_clean[df_clean['ListingId']==10217568]


#To have tuples with LitingsId as index ,useful for later processing
df_tuple = df_clean.set_index('ListingId')
# To compare Harversine we need lat and ln in a tuple
df_clean['Lat_Lng'] =list(zip(df_clean.Lat, df_clean.Lng))

In [ ]:
#Read the feature embeddings df
df_uns = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_uns_label_img_red_hot.csv')
#Stratified random sampling needs binning the agg score of both classes
df_uns_high = df_uns[df_uns['agg_score'] > 0.50]
df_uns_low = df_uns[df_uns['agg_score'] < 0.50]

In [ ]:
# Indexes which may be selected
top_pairs = list(zip(df_uns_high['ListingId_1'],df_uns_high['ListingId_2']))
bottom_pairs = list(zip(df_uns_low['ListingId_1'],df_uns_low['ListingId_2']))

In [ ]:
#Sampling
rng =  np.random.default_rng(seed=2023)
top_idx =  rng.integers(low=0, high=len(top_pairs), size=100)
bottom_idx = rng.integers(low=0, high=len(bottom_pairs), size=100)
#convert to arrays to select with array index
top_pairs = np.array(top_pairs)
bottom_pairs = np.array(bottom_pairs)
#Slect random chosen index
sample_top = top_pairs[top_idx]
sample_bottom = bottom_pairs[bottom_idx]

In [ ]:
df_clean_1 = df_clean.drop(['Host_picture_url','LaBSE_emb', 'Lat_Lng','Host_thumbnail_url'],axis=1)
labels = []

In [ ]:
#Labeling loop
idx = 0
for id_1,id_2 in sample_top:
    print('Label:',idx)
    url_1 =[x for x in df_clean_1['listing_url'][df_clean_1['ListingId'] == id_1]]
    url_2 =[x for x in df_clean_1['listing_url'][df_clean_1['ListingId'] == id_2]]
    print("Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping")
    print(url_1[0])
    print(url_2[0])
    label = int(input())
    if label == 1 or label==0:
        labels.append(((id_1,id_2),label))
        print('Saved!')
    if label == 9:
        break
    if label == 3:
        lista = [print(x,':',y,'==',z) for x,y,z in zip(df_clean_1.columns.tolist(),
                                                        df_clean_1[df_clean_1['ListingId'] == id_1].values.tolist()[0],
                                                        df_clean_1[df_clean_1['ListingId'] == id_2].values.tolist()[0])]
        label = int(input())
        labels.append(((id_1,id_2),label))
        print('Saved!')
    idx += 1

#Labeling loop
idx =0
for id_1,id_2 in sample_bottom:
    print('Label:',idx)
    url_1 =[x for x in df_clean_1['listing_url'][df_clean_1['ListingId'] == id_1]]
    url_2 =[x for x in df_clean_1['listing_url'][df_clean_1['ListingId'] == id_2]]
    print("Use 1 if the urls are of the same property, 0 if not, 3 for a full description, 2 for ambiguous, 9 for stopping")
    print(url_1[0])
    print(url_2[0])
    label = int(input())
    if label == 1 or label==0:
        labels.append(((id_1,id_2),label))
        print('Saved!')
    if label == 9:
        break
    if label == 3:
        lista = [print(x,':',y,'==',z) for x,y,z in zip(df_clean_1.columns.tolist(),
                                                        df_clean_1[df_clean_1['ListingId'] == id_1].values.tolist()[0],
                                                        df_clean_1[df_clean_1['ListingId'] == id_2].values.tolist()[0])]
        label = int(input())
        labels.append(((id_1,id_2),label))
        print('Saved!')
    idx +=1


In [ ]:
# Saving labels as pickle
file_name = "labels_no_hotels.pkl"
open_file = open(file_name, "wb")
pickle.dump(labels, open_file)
open_file.close()